### Bayesian Network Inference Library

We will be using the pomegranate library for Bayes Net inference

  * Installation instructions https://pomegranate.readthedocs.io/en/latest/install.html
  * Tutorial / documentation https://pomegranate.readthedocs.io/en/latest/BayesianNetwork.html
  
In the tutorial / documentation, ignore the parts about "initializing a Bayesian network based completely on data" and the sections on "Probability" "Prediction" and "Fitting" -- see the example below on how to determine the probability distribution on a node in the graph based on evidence.

Just to make sure things are working, first load in the Monty Hall code from the tutorial and answer the question about whether or not a contestant should take Monty up on his offer to switch doors.

The Monty Hall problem:  
  * A prize is placed randomly between door A or door B or door C
  * The guest chooses one of those three doors
  * Monty then reveals whether or not one of the other doors contains the prize
     * Monty will never reveal the door that the guest chooses
     * Monty will nevery reveal the door with the prize
  * Monty gives the guest the chance to change choices to the door that Monty didn't reveal
     
Suppose for example the guest chooses door A.  Monty then reveals that door B does not contain the prize.  Which is the case:

  * This information is of no use to the guest.  It is now equally likely that the prize is behind A and behind C.  Therefore the guest has no incentive to change from A to C
  * It is now more likely that the prize is behind C, so the guest should switch to C
  * It is now more likely that the prize is behind A, so the guest should not switch


In [1]:
from pomegranate import *

# The three "random variables" are
#    guest -- what door will the guest choose -- doors are A, B, and C
#    prize -- what door is the prize behind
#    monty -- what door will Monty open.  This is a function of both guest and prize:
#               Monty will never open the door the guest chooses and will never open the 
#               door with the prize (if the guest doesn't choose it)
#             So the first three lines of the CPT below say the guest chooses A and 
#               the prize is behind A, and then Monty will choose B or C with equal probability

# Notice the pattern of building networks:  
#   -- build your distributions -- either DiscreteDistribution for nodes without parents
#          or ConditionalProbabilityTable for nodes with parents.  The CPT for Monty needs 27 
#          entries, since there are 9 possible combination of parent values, and three possible
#          values the monty random variable can take.

guest = DiscreteDistribution({'A': 1./3, 'B': 1./3, 'C': 1./3})
prize = DiscreteDistribution({'A': 1./3, 'B': 1./3, 'C': 1./3})
monty = ConditionalProbabilityTable(
        [['A', 'A', 'A', 0.0],
         ['A', 'A', 'B', 0.5],
         ['A', 'A', 'C', 0.5],
         ['A', 'B', 'A', 0.0],
         ['A', 'B', 'B', 0.0],
         ['A', 'B', 'C', 1.0],
         ['A', 'C', 'A', 0.0],
         ['A', 'C', 'B', 1.0],
         ['A', 'C', 'C', 0.0],
         ['B', 'A', 'A', 0.0],
         ['B', 'A', 'B', 0.0],
         ['B', 'A', 'C', 1.0],
         ['B', 'B', 'A', 0.5],
         ['B', 'B', 'B', 0.0],
         ['B', 'B', 'C', 0.5],
         ['B', 'C', 'A', 1.0],
         ['B', 'C', 'B', 0.0],
         ['B', 'C', 'C', 0.0],
         ['C', 'A', 'A', 0.0],
         ['C', 'A', 'B', 1.0],
         ['C', 'A', 'C', 0.0],
         ['C', 'B', 'A', 1.0],
         ['C', 'B', 'B', 0.0],
         ['C', 'B', 'C', 0.0],
         ['C', 'C', 'A', 0.5],
         ['C', 'C', 'B', 0.5],
         ['C', 'C', 'C', 0.0]], [guest, prize])

s1 = Node(guest, name="guest")
s2 = Node(prize, name="prize")
s3 = Node(monty, name="monty")

model = BayesianNetwork("Monty Hall Problem")
model.add_states(s1, s2, s3)
model.add_edge(s1, s3)
model.add_edge(s2, s3)
model.bake()

In [2]:
# Based on no more evidence, what is the likelihood that the contestant will win the prize?
model.predict_proba({})

array([{
    "class" :"Distribution",
    "dtype" :"str",
    "name" :"DiscreteDistribution",
    "parameters" :[
        {
            "A" :0.3333333333333333,
            "B" :0.3333333333333333,
            "C" :0.3333333333333333
        }
    ],
    "frozen" :false
},
       {
    "class" :"Distribution",
    "dtype" :"str",
    "name" :"DiscreteDistribution",
    "parameters" :[
        {
            "A" :0.3333333333333333,
            "B" :0.3333333333333333,
            "C" :0.3333333333333333
        }
    ],
    "frozen" :false
},
       {
    "class" :"Distribution",
    "dtype" :"str",
    "name" :"DiscreteDistribution",
    "parameters" :[
        {
            "A" :0.3333333333333333,
            "B" :0.3333333333333333,
            "C" :0.3333333333333333
        }
    ],
    "frozen" :false
}], dtype=object)

In [3]:
# The list above is a list of nodes, or states. How do you get the 
# name of the nodes?   

list(map(lambda s: s.name, model.states))

['guest', 'prize', 'monty']

In [ ]:
model.predict_proba({})[2].parameters

In [4]:
##  Suppose the guest chooses A, and Monty chooses B.
##  Monty gives the guest to switch from A to C.  Should she?

model.predict_proba({"guest": 'A', "monty": 'B'})

array(['A',
       {
    "class" :"Distribution",
    "dtype" :"str",
    "name" :"DiscreteDistribution",
    "parameters" :[
        {
            "A" :0.3333333333333334,
            "B" :0.0,
            "C" :0.6666666666666664
        }
    ],
    "frozen" :false
},
       'B'], dtype=object)

### Second Example:  Typical Noisy Sensor

* The variable **water** says whether or not there is water in my basement.  This variable takes values **{none, some, lots}**
* I have a water detector **waterDetector** that is either **on** or **off**
  * It is supposed to be **on** if and only if **water** is either some or lots
  * However, it sometimes fails to alert (is **off** when **water** is either **some** or **lots**)
  * It also sometimes false alarms (is **on** when **water** is **none**)

This is what I discovered by observing the basement over time
* On any given day, the probability of **water** is **(.98, .015, .005)** for values **(none, some, lots)**
* The likelihood of a false alarm **P(waterDetector = on | water = none) = 0.01**
* The likelihood of the sensor missing water depends on the water level: **P(waterDetector = off | water = some) = .10**;   **P(waterDetector = off | water = lots) = .005**


In [7]:
from pomegranate import *

# This is the distribution for my water node
wdist = DiscreteDistribution({'none': 0.98, 'some': 0.015, 'lots': 0.005})

# Distribution for my water detector node
detectordist = ConditionalProbabilityTable(
    [['none', 'on', 0.01],
     ['none', 'off', 0.99],
     ['some', 'on', 0.9],
     ['some', 'off', 0.1],
     ['lots', 'on', .995],
     ['lots', 'off', .005]],
     [wdist])

#  My two nodes
water = Node(wdist, name="water")
waterDetector = Node(detectordist, name="water detector")

# The Network
model = BayesianNetwork("Water Detector")
model.add_states(water, waterDetector)
model.add_edge(water, waterDetector)
model.bake()
                                     

In [8]:
model.predict_proba({})

array([{
    "class" :"Distribution",
    "dtype" :"str",
    "name" :"DiscreteDistribution",
    "parameters" :[
        {
            "none" :0.9799999999999991,
            "some" :0.015000000000000421,
            "lots" :0.005000000000000436
        }
    ],
    "frozen" :false
},
       {
    "class" :"Distribution",
    "dtype" :"str",
    "name" :"DiscreteDistribution",
    "parameters" :[
        {
            "on" :0.028275000000000623,
            "off" :0.9717249999999994
        }
    ],
    "frozen" :false
}], dtype=object)

With no further information, what is the likelihood that (a) there is some or lots of water in my basement, and (b) what is the likelihood that my water detector is displaying ON

In [ ]:
# Compute probabilities on the basis of no additional evidence.  Its output is a list of 
# distributions over node values, in the order they were added -- in our case, water is at [0] and waterDetector is at [1]
model.predict_proba({})

In [9]:
# This is the distribution over values of water -- not surprising, it is the same as the priors 
#  (subject to rounding error)
model.predict_proba({})[0].parameters

[{'none': 0.9799999999999991,
  'some': 0.015000000000000421,
  'lots': 0.005000000000000436}]

In [ ]:
# This is the distribution over values of waterDetector
model.predict_proba({"water Detector": "on"})

Suppose I learn that the water detector is **on**.  How does that affect my beliefs over the basement water level

In [10]:
model.predict_proba({"water detector": "on"})[0].parameters

[{'none': 0.34659593280282863,
  'some': 0.4774535809018497,
  'lots': 0.17595048629532176}]

Suppose instead I go to the basement and observe that there is no water in the basement.  
How does that affect my belief as to whether or not the water detector is on?

In [12]:
model.predict_proba({"water": "lots"})

array(['lots',
       {
    "class" :"Distribution",
    "dtype" :"str",
    "name" :"DiscreteDistribution",
    "parameters" :[
        {
            "on" :0.9949999999999998,
            "off" :0.005000000000000222
        }
    ],
    "frozen" :false
}], dtype=object)